In [29]:
from llama_cpp import Llama
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain_community.document_loaders import PyPDFLoader

In [ ]:
llm = Llama(
    model_path="DISLab.SummLlama3.2-3B.Q6_K.gguf",
    # n_gpu_layers = 40,
    n_ctx=100000,
    n_batch=1024,
    # main_gpu=0
)

In [30]:
loader = PyPDFLoader("sample.pdf")
pages = loader.load_and_split()
text_splitter = RecursiveCharacterTextSplitter(chunk_size=250, chunk_overlap=50)
texts = text_splitter.split_documents(pages)
final_text = ""
for text in texts:
    if text.page_content.startswith("REFERENCES"):
        break
    else:
        final_text = final_text + text.page_content
research_paper = ""
for text in final_text:
    if text.startswith(("REFERENCES", "REFERENCESREFERENCES", "REFERENCESREFERENCESREFERENCES")):
        break
    else:
        research_paper = research_paper + text

In [ ]:
response = llm.create_chat_completion(
    messages = [
        {'role':'system',
         'content': 'You are a helpful research assistant for generating summary of research papers in about 200-250 words. The output must be in MarkDown format.'},
        {'role':'user',
        'content': research_paper}
    ],
    temperature=0.8,
    top_k=500,
    top_p=3.0,
    presence_penalty = 1.0,
    frequency_penalty = 1.1,
    repeat_penalty=5.0,
)
mindmap_data = response['choices'][0]['message']['content']
print(mindmap_data)